In [ ]:
import os
import json
import pandas as pd
import traceback
from dotenv import load_dotenv
import os

In [ ]:
load_dotenv()
#get the OpenAI API key from environment variables  
openai_api_key = os.getenv("OPENAI_API_KEY") 

In [19]:
from langchain.chat_models import ChatOpenAI
from  langchain import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2

In [20]:
llm= ChatOpenAI(
    model_name="gpt-3.5-turbo",
    temperature=0.5,
    #max_tokens=1000,
    openai_api_key=openai_api_key
)

In [22]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [21]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

In [23]:
quitz_generation_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response_json"],
    template=TEMPLATE
)   

In [24]:
quitz_chain= LLMChain(
    llm=llm,
    prompt=quitz_generation_prompt,
    verbose=True,
    output_key="quitz",
)   

C:\Users\Murali Sidda\AppData\Local\Temp\ipykernel_10916\320513702.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  quitz_chain= LLMChain(


In [25]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [27]:
quitz_evaluation_prompt = PromptTemplate(
    input_variables=[ "subject", "quiz"],
    template=TEMPLATE2
)   

In [ ]:
review_chain= LLMChain(
    llm=llm,
    prompt=quitz_evaluation_prompt,
    verbose=True,
    output_key="review",
)

In [ ]:
#Till now we have created the quiz and the review chain, now we need to create a sequential chain to run both of them in sequence
sequential_chain = SequentialChain(
    chains=[quitz_chain, review_chain],
    input_variables=["text", "number", "subject", "tone", "response_json"],
    output_variables=["quitz", "review"],
    verbose=True,
)       
